In [7]:
import gzip
import struct
import numpy as np

import numdifftools as nd
def softmax_loss(Z, y):
    return np.average([*map(lambda idx:np.log(np.sum(np.exp(Z[idx])))-Z[idx][y[idx]],[x for x in range(len(y))])])

In [12]:
np.random.seed(0)
X = np.random.randn(10,5).astype(np.float32)
y = np.random.randint(3, size=(10,)).astype(np.uint8)
Theta = np.zeros((5,3), dtype=np.float32)

In [14]:
print(y)

[1 1 1 0 1 1 1 0 1 2]


In [9]:
def parse_mnist(image_filename, label_filename):
    """ Read an images and labels file in MNIST format.  See this page:
    http://yann.lecun.com/exdb/mnist/ for a description of the file format.

    Args:
        image_filename (str): name of gzipped images file in MNIST format
        label_filename (str): name of gzipped labels file in MNIST format

    Returns:
        Tuple (X,y):
            X (numpy.ndarray[np.float32]): 2D numpy array containing the loaded 
                data.  The dimensionality of the data should be 
                (num_examples x input_dim) where 'input_dim' is the full 
                dimension of the data, e.g., since MNIST images are 28x28, it 
                will be 784.  Values should be of type np.float32, and the data 
                should be normalized to have a minimum value of 0.0 and a 
                maximum value of 1.0. The normalization should be applied uniformly
                across the whole dataset, _not_ individual images.

            y (numpy.ndarray[dtype=np.uint8]): 1D numpy array containing the
                labels of the examples.  Values should be of type np.uint8 and
                for MNIST will contain the values 0-9.
    """
    ### BEGIN YOUR CODE
    def readLabels(filePath=label_filename):
        with gzip.open(filePath, 'rb') as f:
            return [struct.unpack('>II', f.read(8)),np.frombuffer(f.read(),dtype=np.uint8)]
    def readImages(filePath=image_filename):
        with gzip.open(filePath, 'rb') as f:
            [magic,images,rows,cols]=struct.unpack('>IIII', f.read(16))
            return np.resize(np.frombuffer(f.read(),dtype=np.uint8),(images,rows*cols))/np.float32(255)
    return (readImages(),readLabels()[1])

In [10]:
def softmax_loss(Z, y):
    """ Return softmax loss.  Note that for the purposes of this assignment,
    you don't need to worry about "nicely" scaling the numerical properties
    of the log-sum-exp computation, but can just compute this directly.

    Args:
        Z (np.ndarray[np.float32]): 2D numpy array of shape
            (batch_size, num_classes), containing the logit predictions for
            each class.
        y (np.ndarray[np.int8]): 1D numpy array of shape (batch_size, )
            containing the true label of each example.

    Returns:
        Average softmax loss over the sample.
    """
    ### BEGIN YOUR CODE
    return np.average([*map(lambda idx:np.log(np.sum(np.exp(Z[idx])))-Z[idx][y[idx]],[x for x in range(len(y))])])

In [11]:
def softmax_regression_epoch(X, y, theta, lr = 0.1, batch=100):
    """ Run a single epoch of SGD for softmax regression on the data, using
    the step size lr and specified batch size.  This function should modify the
    theta matrix in place, and you should iterate through batches in X _without_
    randomizing the order.

    Args:
        X (np.ndarray[np.float32]): 2D input array of size
            (num_examples x input_dim).
        y (np.ndarray[np.uint8]): 1D class label array of size (num_examples,)
        theta (np.ndarrray[np.float32]): 2D array of softmax regression
            parameters, of shape (input_dim, num_classes)
        lr (float): step size (learning rate) for SGD
        batch (int): size of SGD minibatch

    Returns:
        None
    """
    ### BEGIN YOUR CODE
    def normalize(cur):
        def calCuri(curi): 
            return np.exp(curi)/np.sum(np.exp(cur))
        return [*map(calCuri,cur)]
    for index in range(0,len(y),batch):
        batchX,batchy=X[index:index+batch],y[index:index+batch]
        Z=np.array([*map(normalize,np.dot(batchX,theta))])
        newy=np.array([np.concatenate((x[0:yy],np.array([1]),x[yy+1:]), axis=0) 
          for (x,yy) in zip(np.zeros(shape=(batchy.shape[0],theta[0].shape[0])),batchy)
         ])
        theta-=(lr/batch)*np.dot(batchX.transpose(),(Z-newy))
    ### END YOUR CODE

In [12]:
def nn_epoch(X, y, W1, W2, lr = 0.1, batch=100):
    """ Run a single epoch of SGD for a two-layer neural network defined by the
    weights W1 and W2 (with no bias terms):
        logits = ReLU(X * W1) * W2
    The function should use the step size lr, and the specified batch size (and
    again, without randomizing the order of X).  It should modify the
    W1 and W2 matrices in place.

    Args:
        X (np.ndarray[np.float32]): 2D input array of size
            (num_examples x input_dim).
        y (np.ndarray[np.uint8]): 1D class label array of size (num_examples,)
        W1 (np.ndarray[np.float32]): 2D array of first layer weights, of shape
            (input_dim, hidden_dim)
        W2 (np.ndarray[np.float32]): 2D array of second layer weights, of shape
            (hidden_dim, num_classes)
        lr (float): step size (learning rate) for SGD
        batch (int): size of SGD minibatch

    Returns:
        None
    """
    ### BEGIN YOUR CODE
    def RELU(arr):
        return np.maximum(0, arr)
    def softmax_columns(arr):
        return np.exp(arr) / np.sum(np.exp(arr), axis=1, keepdims=True)
    for index in range(0,len(y),batch):
        batchX,batchy=X[index:index+batch],y[index:index+batch]
        Iy=np.array([np.concatenate((x[0:yy],np.array([1]),x[yy+1:]), axis=0) 
          for (x,yy) in zip(np.zeros(shape=(batchy.shape[0],W2.shape[1])),batchy)
         ])
        Z1=RELU(np.dot(batchX,W1))
        G2=softmax_columns(np.dot(Z1,W2))-Iy
        G1=np.multiply(np.where(Z1 > 0, 1, 0),(np.dot(G2,W2.transpose())))
        W1-=(lr/batch)*np.dot(batchX.transpose(),G1)
        W2-=(lr/batch)*np.dot(Z1.transpose(),G2)

    ### END YOUR CODE

In [13]:
def train_nn(X_tr, y_tr, X_te, y_te, hidden_dim = 500,
             epochs=10, lr=0.5, batch=100):
    """ Example function to train two layer neural network """
    n, k = X_tr.shape[1], y_tr.max() + 1
    np.random.seed(0)
    W1 = np.random.randn(n, hidden_dim).astype(np.float32) / np.sqrt(hidden_dim)
    W2 = np.random.randn(hidden_dim, k).astype(np.float32) / np.sqrt(k)

    print("| Epoch | Train Loss | Train Err | Test Loss | Test Err |")
    for epoch in range(epochs):
        nn_epoch(X_tr, y_tr, W1, W2, lr=lr, batch=batch)
        train_loss, train_err = loss_err(np.maximum(X_tr@W1,0)@W2, y_tr)
        test_loss, test_err = loss_err(np.maximum(X_te@W1,0)@W2, y_te)
        print("|  {:>4} |    {:.5f} |   {:.5f} |   {:.5f} |  {:.5f} |"\
              .format(epoch, train_loss, train_err, test_loss, test_err))
def loss_err(h,y):
    """ Helper funciton to compute both loss and error"""
    return softmax_loss(h,y), np.mean(h.argmax(axis=1) != y)

In [14]:
X_tr, y_tr = parse_mnist("data/train-images-idx3-ubyte.gz", 
                         "data/train-labels-idx1-ubyte.gz")
X_te, y_te = parse_mnist("data/t10k-images-idx3-ubyte.gz",
                         "data/t10k-labels-idx1-ubyte.gz")
train_nn(X_tr, y_tr, X_te, y_te, hidden_dim=400, epochs=20, lr=0.2)

| Epoch | Train Loss | Train Err | Test Loss | Test Err |
|     0 |    0.15324 |   0.04697 |   0.16305 |  0.04920 |
|     1 |    0.09854 |   0.02923 |   0.11604 |  0.03660 |
|     2 |    0.07429 |   0.02168 |   0.09774 |  0.03160 |
|     3 |    0.05959 |   0.01732 |   0.08790 |  0.02930 |
|     4 |    0.04820 |   0.01348 |   0.08064 |  0.02610 |
|     5 |    0.04016 |   0.01087 |   0.07663 |  0.02380 |
|     6 |    0.03446 |   0.00897 |   0.07410 |  0.02330 |
|     7 |    0.03023 |   0.00765 |   0.07252 |  0.02260 |
|     8 |    0.02660 |   0.00650 |   0.07101 |  0.02240 |
|     9 |    0.02348 |   0.00545 |   0.06992 |  0.02160 |
|    10 |    0.02099 |   0.00472 |   0.06905 |  0.02160 |
|    11 |    0.01891 |   0.00392 |   0.06836 |  0.02130 |
|    12 |    0.01702 |   0.00328 |   0.06769 |  0.02090 |
|    13 |    0.01559 |   0.00282 |   0.06728 |  0.02110 |
|    14 |    0.01406 |   0.00240 |   0.06667 |  0.02070 |
|    15 |    0.01276 |   0.00208 |   0.06602 |  0.02020 |
|    16 |    0

In [15]:
def softmax_columns(arr):
    return np.exp(arr) / np.sum(np.exp(arr), axis=1, keepdims=True)

In [16]:
arr = np.array([[1, 2, 3], [4, 5, 6]])
softmax_columns(arr)

array([[0.09003057, 0.24472847, 0.66524096],
       [0.09003057, 0.24472847, 0.66524096]])